## Need to run:
coevolve python package (currently only installed locally in /data2/isshamie/coevolve/coevolve. Go to that folder and run pip install -e .)

1. parameters folder. The evc template is in there and the parameter file that points to the genome information (in case that changes).  
2. data/raw is the folder that contains that genome files  
3. data/notebook with this file and Covid19 - Alignment and Couplings Output.ipynb

### Parameter file (should be in parameters folder)

In [1]:
param = 'sars_cov2.yaml'

### 0.1 Import 

In [2]:
import pandas as pd
import os
import glob
from Bio import SeqIO, Seq, pairwise2, SeqRecord, Align
from os.path import join, basename
import pickle
import numpy as np
from tqdm import tqdm
from numpanpar import parallel_ar as parar

from coevolve.msa import clustalw_query
from coevolve.config import CLUSTALW_DIR, CLUSTALW_CMD, NUM_CORES, CLUSTALW_CLEAN_DIR
from evcouplings.utils import read_config_file, write_config_file
from coevolve.couplings.running_jobs_complexes import run_evc
from coevolve.couplings.couplings_vis import construct_monomer_contact_matrix

from config import RESULTS, CONTACT_FIGURES_DIR
%load_ext autoreload
%autoreload 2

/home/isshamie/software/anaconda2/envs/coevolve/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Project Directory: /data2/isshamie/coevolve

Config paths:
__name__ coevolve.config
__package__ coevolve
__file__ /data2/isshamie/coevolve/coevolve/config.py
__cached__ /data2/isshamie/coevolve/coevolve/__pycache__/config.cpython-37.pyc
path /data2/isshamie/coevolve/coevolve/config.py
DATA_DIR /data2/isshamie/coevolve/data
RESULTS /data2/isshamie/coevolve/data/processed
FIGURES_DIR /data2/isshamie/coevolve/figures
FIG_NET_DIR /data2/isshamie/coevolve/figures/networks
PIPELINE_F /data2/isshamie/coevolve/reports/pipeline/Pipeline.xlsx
DATA_FIG_DIR /data2/isshamie/coevolve/data/processed/figures
PARAMS_DIR /data2/isshamie/coevolve/params
TEST_DIR /data2/isshamie/coevolve/tests
ORTHO_FIGURES_DIR /data2/isshamie/coevolve/figures/orthologs
ORTHO_SEQS_DIR /data2/isshamie/coevolve/data/external/orthodb/protein_sequences/
ODB_DIR /data2/isshamie/coevolve/data/external/orthodb/
TAXA_ID_TO_NAME_RAW /data2/isshamie/coevolve/data/external/ncbi_taxonomy/names.dmp
NCBI_FOLDER /data2/isshamie/coevolve

In [3]:
from config import FIGURES_DIR, ROOT_DIR, PARAMS_DIR

### Setup folders and files

In [4]:
RAW = join(ROOT_DIR,"data","raw")
outdir = join(ROOT_DIR,"data","processed")


seqs_dir = join(outdir,"seqs")
align_dir = join(seqs_dir,"S_protein_align")
align_concat_dir = join(seqs_dir,"S_protein_align_full")
msa_out = join(outdir, "msa")
anno_f = join(outdir, "covid_annotations.csv")
ev_dir = join(outdir,"evc")
contact_dir = join(outdir, "contact")
ev_params_dir = join(outdir,"ev_params")
evzoom_dir = join(outdir,"evzoom")
evzoom_template_dir = join(ROOT_DIR, "software", "EVzoom")
param_dir = PARAMS_DIR
evc_template_f = join(param_dir,'evc',"template.yaml")

msa_f = join(msa_out,"S_protein.msa.fasta")
anno_f = join(seqs_dir, "covid_annotations.csv")
clean = join(msa_out,"clean")
if not os.path.exists(clean):
    os.mkdir(clean)

#clustalw_query.create_clean_clustalw_names(clean, [msa_f])

msa_clean_f = join(clean, os.path.basename(msa_f))

NUM_CORES = 32
if not os.path.exists(outdir):
    os.mkdir(outdir)

if not os.path.exists(seqs_dir):
    os.mkdir(seqs_dir)
if not os.path.exists(align_dir):
    os.mkdir(align_dir)
    
if not os.path.exists(align_concat_dir):
    os.mkdir(align_concat_dir)
    
if not os.path.exists(msa_out):
    os.mkdir(msa_out)
    
if not os.path.exists(ev_dir):
    os.mkdir(ev_dir)
    
if not os.path.exists(ev_params_dir):
    os.mkdir(ev_params_dir)
    
if not os.path.exists(param_dir):
    os.mkdir(param_dir)
    
if not os.path.exists(evzoom_dir):
    cmd = f'cp -r {evzoom_template_dir} {evzoom_dir}'
    print(cmd)
    os.system(cmd)

if not os.path.exists(contact_dir):
    os.mkdir(contact_dir)
    

## 1. Load:
- A) Genomic fasta sequences, 
- B) Reference Genomic Annotation (GCF_009858895.2)
- C) Reference Gene Annotation (NC_045512.2)
- D) S-protein fasta protein sequence

In [5]:
config = read_config_file(join(param_dir, param))
config

{'ref_fasta': 'NC_045512.2.fasta',
 'S_protein_name': 'YP_009724390.1',
 'annotation': 'GCF_009858895.2_ASM985889v3_genomic.gff'}

In [6]:
# ref_fasta = SeqIO.read(os.path.join(RAW,'NC_045512.2.fasta'), format="fasta")
# anno = pd.read_csv(glob.glob(RAW+"/*.gff")[0],sep="\t", comment="#", header=None)
# all_genomic_sequences = list(SeqIO.parse(os.path.join(RAW,"gisaid_cov2020_sequences.fasta"), "fasta"))

# S_protein_name = "YP_009724390.1"
# S_protein_fasta = SeqIO.read(os.path.join(RAW, S_protein_name+".fasta"),format='fasta')
# S_protein_fasta = str(S_protein_fasta.seq)


ref_fasta = SeqIO.read(os.path.join(RAW,config['ref_fasta']), format="fasta")
anno = pd.read_csv(join(RAW,config['annotation']),sep="\t", comment="#", header=None)
all_genomic_sequences = list(SeqIO.parse(os.path.join(RAW,"gisaid_cov2020_sequences.fasta"), "fasta"))

S_protein_name = config['S_protein_name']
S_protein_fasta = SeqIO.read(os.path.join(RAW, S_protein_name+".fasta"),format='fasta')
S_protein_fasta = str(S_protein_fasta.seq)


## 2. Preprocess

### a. Filter annotation for the S-protein.
Confirm the protein sequence aligns with the genome annotation.
- Extract the genomic region from annotation and take the sequence
- Translate sequence
- See if aligns with the Protein fasta sequence



See below that there are no introns,  so we can just use that sequence

In [7]:
anno_S = anno[anno[8].str.contains(S_protein_name)]

#Extract protein sequence from this region
S_gene_seq = ref_fasta.seq[anno_S[3].values[0]-1: anno_S[4].values[0]]
annotation_protein_sequence = str(Seq.translate(S_gene_seq))
annotation_protein_sequence = annotation_protein_sequence.replace("*","")

S_gene_seq = SeqRecord.SeqRecord(S_gene_seq, id=S_protein_name)
SeqIO.write(S_gene_seq, open(join(seqs_dir,"S_gene.fasta"),'w'),format='fasta')

1

In [8]:
annotation_protein_mismatch = []
for ind, val in enumerate(annotation_protein_sequence):
    if S_protein_fasta[ind] != val:
        annotation_protein_mismatch.append((ind, val))
annotation_protein_mismatch

[]

### b. remove any sequence less than 20kb

In [9]:
print(f"Number of sequences beforeM filtering: {len(all_genomic_sequences)}")
all_genomic_sequences_filt = []
for seq in all_genomic_sequences:
    if len(seq.seq) >= 20000:
        all_genomic_sequences_filt.append(seq)
print(f"Number of sequences after filtering: {len(all_genomic_sequences_filt)}")

Number of sequences beforeM filtering: 664
Number of sequences after filtering: 563


## 3. Local Alignment each sequence to the genome using Bio.pairwise2.align.localxx
## and save the alignments and the translated sequence 

In [10]:
def remove_chars(orig_str, chars = ["|",")","(","*"]):
    return orig_str.replace("|","_").replace(")","").replace("*","_").replace("(","").replace(" ","_").replace("-","_").replace("/","").replace(">","")

In [11]:
def run_local(seqs, S_gene_seq=S_gene_seq, save_dir=align_dir,to_overwrite=False):
    if not os.path.exists(save_dir):
        print(f"{save_dir} not made. Need to create it")
        return
    invalid_seqs = []
    S_align_full = dict()
    S_align_first = dict()
    S_align_first_seqs = dict() #Dictionary of the subseqeunce of the genome that locally aligned to the S gene.
    S_all_proteins = []
    count = 0
    
    for seq in tqdm(seqs):
        try:
            seq.seq= seq.seq.upper()
            seq.seq = str(seq.seq).replace("-","")
            name = remove_chars(seq.id)
            curr_out = pairwise2.align.localms(str(seq.seq),str(S_gene_seq.seq), 2, -10, -0.5, -0.1)#, one_alignment=True)
            curr_align = str(seq.seq)[curr_out[0][3]:curr_out[0][4]]

            S_all_proteins = [(SeqRecord.SeqRecord(Seq.Seq(Seq.translate(curr_align, stop_symbol="")), id=name))]
            print('s all proteins', S_all_proteins)
            print('to save', join(save_dir,name+".protein.fasta"))
            pickle.dump([curr_out,str(seq.seq)[curr_out[0][3]:curr_out[0][4]],], open(join(save_dir, name+".p"),"wb"))
            SeqIO.write(S_all_proteins, open(join(save_dir,name+".protein.fasta"),"w"),format="fasta")
        except TypeError:
            with open(join(save_dir, 'invalid_' + name + '.txt'),'w') as f:
                f.write("")
    return

In [12]:
parar(np.array(all_genomic_sequences_filt),run_local, num_processes=32)

  0%|          | 0/17 [00:00<?, ?it/s]

s all proteins [SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='hCoV_19ChinaWHU022020_EPI_ISL_406717_2020_01_02', name='<unknown name>', description='<unknown description>', dbxrefs=[])]
to save /data2/isshamie/covid/data/processed/seqs/S_protein_align/hCoV_19ChinaWHU022020_EPI_ISL_406717_2020_01_02.protein.fasta


  6%|▌         | 1/17 [02:56<47:02, 176.39s/it]/home/isshamie/software/anaconda2/envs/coevolve/lib/python3.7/site-packages/Bio/Seq.py:2859: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


s all proteins [SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='hCoV_19USAIL12020_EPI_ISL_404253_2020_01_21', name='<unknown name>', description='<unknown description>', dbxrefs=[])]
to save /data2/isshamie/covid/data/processed/seqs/S_protein_align/hCoV_19USAIL12020_EPI_ISL_404253_2020_01_21.protein.fasta


  6%|▌         | 1/17 [02:56<47:07, 176.74s/it]

...

100%|██████████| 17/17 [1:03:31<00:00, 340.89s/it]


s all proteins [SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='hCoV_19USAWA_UW322020_EPI_ISL_414619_2020_03_07', name='<unknown name>', description='<unknown description>', dbxrefs=[])]
to save /data2/isshamie/covid/data/processed/seqs/S_protein_align/hCoV_19USAWA_UW322020_EPI_ISL_414619_2020_03_07.protein.fasta


 94%|█████████▍| 16/17 [25:14<01:47, 107.20s/it]

s all proteins [SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='hCoV_19USAWA_UW332020_EPI_ISL_414620_2020_03_08', name='<unknown name>', description='<unknown description>', dbxrefs=[])]
to save /data2/isshamie/covid/data/processed/seqs/S_protein_align/hCoV_19USAWA_UW332020_EPI_ISL_414620_2020_03_08.protein.fasta


100%|██████████| 17/17 [26:32<00:00, 98.43s/it] 


In [13]:
S_protein_sequences_f = join(align_concat_dir,"S_protein.fasta")
cmd = f'cat {join(align_dir,"*fasta")} > {S_protein_sequences_f}'
print(cmd)
os.system(cmd)

cat /data2/isshamie/covid/data/processed/seqs/S_protein_align/*fasta > /data2/isshamie/covid/data/processed/seqs/S_protein_align_full/S_protein.fasta


0

## 5. Plot similarity

## 6. Run MSA on protein using clustalw

In [14]:
clustalw_query.extract_msa(S_protein_sequences_f,msa_out, CLUSTALW_CMD, "isshamie@ucsd.edu",
                to_overwrite=False, web=True,
                local_backup=True, num_cores=NUM_CORES, verbose=True)

S_protein
/home/isshamie/software/anaconda2/envs/coevolve/bin/python /data2/isshamie/coevolve/software/clustalo.py --email isshamie@ucsd.edu --stype protein --outfmt fa --outfile /data2/isshamie/covid/data/processed/msa/S_protein.msa.fasta --sequence  /data2/isshamie/covid/data/processed/seqs/S_protein_align_full/S_protein.fasta
mv /data2/isshamie/covid/data/processed/msa/S_protein.msa.fasta.aln-fasta.fasta /data2/isshamie/covid/data/processed/msa/S_protein.msa.fasta


## 7. Run EVCouplings
- Create annotation and yaml
- Run 

In [36]:

record_dict = SeqIO.to_dict(SeqIO.parse(msa_f, "fasta"))

anno_df = pd.DataFrame(index=record_dict.keys(),columns=["id", "name", "GN", "OS", "Tax", "RepID",
                       "id_raw"],dtype=str)
for ind, val in anno_df.iterrows():
    val[:] = ind
anno_df.to_csv(anno_f, index=None)


In [16]:
def create_annotation_from_fasta(msa_f, anno_f, include_region=False):
    """Create a csv file where each sequence is considered its own organism. 
    Columns needed are ["id", "name", "GN", "OS", "Tax", "RepID",
                       "id_raw"] """
    record_dict = SeqIO.to_dict(SeqIO.parse(msa_f, "fasta"))
    anno_df = pd.DataFrame(index=record_dict.keys(),columns=["id", "name", "GN", "OS", "Tax", "RepID",
                           "id_raw"],dtype=str)
    for ind, val in anno_df.iterrows():
        val[:] = ind
    anno_df.to_csv(anno_f, index=None)
    return

# def run_yaml():
#     configs = [os.path.join(outdir,  "covid.yaml")]
#     print(configs)
#     run_evc(configs, to_overwrite=True)
#     return


In [17]:
#create_annotation_from_fasta(msa_f, anno_f)

## Run with both high coverage of a residue and low 
### Need a fake target sequence, so take the first one, but make sure the minimum_sequence_coverage is 0

In [28]:
cmd = f"sed 's/\s.*$//' {msa_f} > {msa_clean_f}"
print(cmd)
os.system(cmd)

sed 's/\s.*$//' /data2/isshamie/covid/data/processed/msa/S_protein.msa.fasta > /data2/isshamie/covid/data/processed/msa/clean/S_protein.msa.fasta


0

In [ ]:
## 
column_thresholds = [90,75,25]
redundant_thresholds = [0.8,0.9]

files_to_run = []
for col_thresh in column_thresholds:
    for red_thresh in redundant_thresholds:
        name = f"col{col_thresh}_redund{red_thresh}"
        config = read_config_file(evc_template_f, preserve_order=True)
        config["align"]["input_alignment"] = msa_clean_f
        config["align"]["override_annotation_file"] = anno_f
        seqid = pd.read_csv(anno_f).iloc[100,0] #Just take any name
        print(seqid) #clean
        config["align"]["sequence_id"] = seqid
        config["global"]["sequence_id"] = seqid
        config["global"]["sequence_file"] = msa_clean_f
        config["global"]["prefix"] = os.path.join(ev_dir, name)
        config["align"]["minimum_sequence_coverage"] = 0
        config["align"]["minimum_column_coverage"] = col_thresh
        config["align"]["theta"] = red_thresh
        config["global"]["theta"] = red_thresh
        out_f = join(ev_params_dir,name+".yaml")
        files_to_run.append(out_f)
        print(f"Creating file {out_f}")
        write_config_file(out_f, config)
        
run_evc(files_to_run)

hCoV_19FranceIDF19802020_EPI_ISL_414633_2020_03_04
Creating file /data2/isshamie/covid/data/processed/ev_params/col90_redund0.8.yaml
hCoV_19FranceIDF19802020_EPI_ISL_414633_2020_03_04
Creating file /data2/isshamie/covid/data/processed/ev_params/col90_redund0.9.yaml
hCoV_19FranceIDF19802020_EPI_ISL_414633_2020_03_04
Creating file /data2/isshamie/covid/data/processed/ev_params/col75_redund0.8.yaml
hCoV_19FranceIDF19802020_EPI_ISL_414633_2020_03_04
Creating file /data2/isshamie/covid/data/processed/ev_params/col75_redund0.9.yaml
hCoV_19FranceIDF19802020_EPI_ISL_414633_2020_03_04
Creating file /data2/isshamie/covid/data/processed/ev_params/col25_redund0.8.yaml
hCoV_19FranceIDF19802020_EPI_ISL_414633_2020_03_04
Creating file /data2/isshamie/covid/data/processed/ev_params/col25_redund0.9.yaml
/data2/isshamie/covid/data/processed/ev_params/col90_redund0.8.yaml
/data2/isshamie/covid/data/processed/evc/col90_redund0.8
{}
HI


/data2/isshamie/coevolve/software/EVcouplings/evcouplings/visualize/parameters.py:137: RuntimeWarning: divide by zero encountered in log2
  B = -fi * np.log2(fi)
/data2/isshamie/coevolve/software/EVcouplings/evcouplings/visualize/parameters.py:137: RuntimeWarning: invalid value encountered in multiply
  B = -fi * np.log2(fi)


/data2/isshamie/covid/data/processed/ev_params/col90_redund0.9.yaml
/data2/isshamie/covid/data/processed/evc/col90_redund0.9
{}
HI
/data2/isshamie/covid/data/processed/ev_params/col75_redund0.8.yaml
/data2/isshamie/covid/data/processed/evc/col75_redund0.8
{}
HI


## Make EVZoom htmls and move jsons

In [ ]:
column_thresholds = [90,75,25]
redundant_thresholds = [0.8,0.9]

files_to_run = []
for col_thresh in column_thresholds:
    for red_thresh in redundant_thresholds:
        gene_of_interest = f"col{col_thresh}_redund{red_thresh}"
        print(gene_of_interest)
        #Copy over json
        coupling_prefix = os.path.join(evc_dir, gene_of_interest,"couplings",gene_of_interest)
        json_f = coupling_prefix + '_evzoom.json'
        out_json_f = evzoom_dir+"/data/" + os.path.basename(json_f)
        cmd = f'cp {json_f} {out_json_f}'
        print(cmd)
        os.system(cmd)
        
        # Create the html file
        tmp_html = join(evzoom_dir, 'example','evzoom.html')
        out_html = join(evzoom_dir, 'example',gene_of_interest+'.html')
        print(out_html)
        with open(tmp_html, "r") as f:
            page = f.read()
        new_page=""
        line = f'<div id="evzoom-viewer" data-couplings=../data/{basename(json_f)}> </div> '
        for i in page.split("\n"):
            if 'data-couplings' in i:
                new_page = new_page + line + "\n"
            else:
                new_page = new_page + i + "\n"
        new_page = new_page.strip("\n")
        with open(out_html,'w') as f:
            f.write(new_page)


## Create contact matrix for cn and probability

In [ ]:
## 
column_thresholds = [90,75,25]
redundant_thresholds = [0.8,0.9]

files_to_run = []
for col_thresh in column_thresholds:
    for red_thresh in redundant_thresholds:
        gene_of_interest = f"col{col_thresh}_redund{red_thresh}"
        print(gene_of_interest)
        align_prefix = os.path.join(evc_dir,gene_of_interest,"align",gene_of_interest)
        coupling_prefix = os.path.join(evc_dir, gene_of_interest,"couplings",gene_of_interest)

        construct_monomer_contact_matrix(gene_of_interest, 
                                         input_dir = evc_dir, 
                                         to_plot=True, 
                                         f_save_fig = join(FIGURES_DIR, "contact", gene_of_interest),
                                         f_save = join(contact_dir, gene_of_interest), 
                                         col='cn', conservation=True) 
        construct_monomer_contact_matrix(gene_of_interest, 
                                         input_dir = evc_dir, 
                                         to_plot=True, 
                                         f_save_fig = join(FIGURES_DIR, "contact", gene_of_interest+"_pval"),
                                         f_save = join(contact_dir, gene_of_interest+"_pval"), 
                                         col='probability', conservation=True) 